In [ ]:
import xgeo.io
import rasterio
import xarray as xr
from os.path import join, basename

In [ ]:
# read geo coords from params file
def _read_params(fn, col_width=12):
    rename = {'grid number (north-south)': 'height',
              'grid number (east-west)': 'width',
              'west  edge [deg]': 'west',
              'north edge [deg]': 'north',
              'grid size  [deg]': 'res'}
    with open(fn, 'r') as txt:
        params = {line[col_width:].strip(): float(line[:col_width].strip())
                    for line in txt.readlines()}
    params = {rename[key]: params[key] for key in params if key in rename.keys()}
    transform = rasterio.transform.from_origin(params['west'], params['north'], params['res'], params['res'])
    return transform, (int(params['height']), int(params['width']))

In [ ]:
ddir = r'd:\temp\glofrim\Amazon\WFL2CMF\CMF'
transform, (nrow, ncol) = _read_params(join(ddir, 'params.txt'))


In [ ]:
prefix = ['fldsto', 'flddph', 'outflw', 'rivout']
dlist = []
for p in prefix:
    da = xgeo.io.from_mfbin(ddir, p, nrow, ncol, transform=transform, nodata=1e+20).load()
    dlist.append(da)
xr.merge(dlist).to_netcdf(join(ddir, 'WFL2CMF_Amazon.nc'), encoding={d.name: {'zlib': True} for d in dlist})

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

da.isel(time=0).plot()